**Problem:**

to determine the extent to which inflation rates in India and the United States influence the INR/USD exchange rates over time.

1. identify the correlation between exchange rates and inflation rates, considering potential lag effects.
2. analyze whether purchasing power parity (PPP) theory holds by comparing actual exchange rate with expected exchange rate based on inflation differentials.
3. eplore any deviations from the expected PPP-based exchange rate to uncover other economic factors that may contribute to exchange rate fluctuations

In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objs as go

inflation_data = pd.read_csv('/content/Inflation_Rates_Transformed-1.csv')
exchange_rate_data = pd.read_csv('/content/USD_INR_Exchange_Rates_1980_2024.csv')

exchange_rate_data.head()

,Year,Average Exchange Rate (INR/USD)
0,2003,45.476087
1,2004,45.188604
2,2005,44.003753
3,2006,45.183561
4,2007,41.192248


In [2]:
inflation_data.head()

,Country,Year,Inflation Rate
0,India,1980,11.3
1,United States,1980,13.5
2,Emerging market and developing economies,1980,26.0
3,India,1981,12.7
4,United States,1981,10.4


In [3]:
# filter the inflation data for India and United States
inflation_filtered_df = inflation_data[inflation_data['Country'].isin(['India', 'United States'])]

# pivot the inflation data to have separate columns for India and United States Inflation Rates
inflation_pivoted_df = inflation_filtered_df.pivot(index='Year', columns='Country', values='Inflation Rate').reset_index()

# merge the exchange rates data with inflation data
merged_df = pd.merge(exchange_rate_data, inflation_pivoted_df, on='Year')

# renaming columns
merged_df.columns = ['Year', 'INR/USD Exchange Rate', 'India Inflation Rate', 'US Inflation Rate']

merged_df.head()

,Year,INR/USD Exchange Rate,India Inflation Rate,US Inflation Rate
0,2003,45.476087,4.1,2.3
1,2004,45.188604,4.0,2.7
2,2005,44.003753,3.7,3.4
3,2006,45.183561,6.7,3.2
4,2007,41.192248,5.9,2.9


In [6]:
import plotly.graph_objs as go
from plotly.subplots import make_subplots

fig = make_subplots(rows=3, cols=1, shared_xaxes=True, vertical_spacing=0.1,
                    subplot_titles=('INR/USD Exchange Rate', 'India Inflation Rate', 'US Inflation Rate'))

fig.add_trace(go.Scatter(x=merged_df['Year'], y=merged_df['INR/USD Exchange Rate'],
                         mode='lines+markers', marker=dict(color='blue'), name='INR/USD Exchange Rate'), row=1, col=1)

fig.add_trace(go.Scatter(x=merged_df['Year'], y=merged_df['India Inflation Rate'],
                         mode='lines+markers', marker=dict(color='orange'), name='India Inflation Rate'), row=2, col=1)

fig.add_trace(go.Scatter(x=merged_df['Year'], y=merged_df['US Inflation Rate'],
                         mode='lines+markers', marker=dict(color='green'), name='US Inflation Rate'), row=3, col=1)

fig.update_layout(height=800, width=900, showlegend=False, title_text='Trends of Exchange rates and Inflation Rates',
                  xaxis3_title='Year', template='plotly_white')

fig.update_yaxes(title_text="Exchange Rate (INR/USD)", row=1, col=1)
fig.update_yaxes(title_text="Inflation Rate (%)", row=2, col=1)
fig.update_yaxes(title_text="Inflation Rate (%)", row=3, col=1)

fig.show()

the first graph shows an upward trend due to the depreciation of INR against USD. However there were periods of both sharp increases and relative stability.

the second graph depicts the significant fluctuations in the infltion rates of India. the inflation rates seem tohve stablized in the recent years.

the third graph depicts the stability of inflation rates in US with a sharp increase in recent years

In [7]:
correlation_matrix = merged_df[['INR/USD Exchange Rate', 'India Inflation Rate', 'US Inflation Rate']].corr()

correlation_matrix

,INR/USD Exchange Rate,India Inflation Rate,US Inflation Rate
INR/USD Exchange Rate,1.000000,-0.337748,0.235321
India Inflation Rate,-0.337748,1.000000,-0.120503
US Inflation Rate,0.235321,-0.120503,1.000000


1. Exchange rate vs india inflation rate (-0.34): as inflation rate in india increases INR tends to depreciate against USD
2. Exchange rate vs US inflation rate (0.23): higher inflationrate in US is due to depreciation of USD against INR
3. US inflation rate vs indian inflation rate (-0.12): inflation does not move together

In [8]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=merged_df['Year'], y=merged_df['INR/USD Exchange Rate'],
              mode='lines+markers', marker=dict(color='blue')))

fig.add_trace(go.Scatter(x=merged_df['Year'], y=merged_df['India Inflation Rate'],
              mode='lines+markers', marker=dict(color='orange')))

fig.add_trace(go.Scatter(x=merged_df['Year'], y=merged_df['US Inflation Rate'],
              mode='lines+markers', marker=dict(color='green')))

fig.update_layout(height=600, width=1000, title_text='Comparative Analysis: Exchange Rate vs Inflation Rates (India & US)',
                  xaxis_title='Year', yaxis_title='Value', legend_title_text='Indicators', template='plotly_white')

fig.show()

1. early 2000s: high inflation did not affect the exchange rate much leading to stable exchange rates
2. late 2000s to 2010s: shows rising inflation rates leading to lower exchange rates in india
3. 2015 onwards: exchange rates rise while inflation rates are relatively low suggesting there are other factors that cause change in exchange rates


formula for PPP-based exchange rate prediction

**Expected Exchange Rate = Initial Exchange Rate × ( 1 + Inflation Rate in India / 1 + Inflation Rate in the US)**

In [13]:
initial_exchange_rate = merged_df['INR/USD Exchange Rate'].iloc[0]

# calculate expected exchange rate based on PPP
merged_df['Expected Exchange Rate'] = initial_exchange_rate * ((1 + merged_df['India Inflation Rate'] / 100) / (1 + merged_df['US Inflation Rate'] / 100)).cumprod()

fig = go.Figure()

# plot actual exchange rate
fig.add_trace(go.Scatter(x=merged_df['Year'], y=merged_df['INR/USD Exchange Rate'],
                         mode='lines+markers', name='Actual Exchange Rate', line=dict(color='blue')))

# plot PPP-based expected exchange rate
fig.add_trace(go.Scatter(x=merged_df['Year'], y=merged_df['Expected Exchange Rate'],
                         mode='lines+markers', name='Expected Exchange Rate', line=dict(color='orange', dash='dash')))

fig.update_layout(height=600, width=1000, title_text='Actual vs Expected Exchange Rate (PPP)',
                  xaxis_title='Year', yaxis_title='INR/USD Exchange Rate', legend_title_text='Exchange Rates', template='plotly_white')

fig.show()

the orange dashed line represents the PPP baed exchange rate. In some periods the actual exchange rate follows the PPP- based rate which means the PPP theory holds.

according to this result the PPP-based exchange rates show that INR should have depreciated more than it actually did. the actual exchange rates shows that some other factors other than inflation rates are at play.